In [73]:
import numpy as np

In [74]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

# Create a test environment for use later
board_size = 10
environment = make("halite", configuration={"size": board_size, "startingHalite": 5000})
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)
me = board.current_player

In [75]:
def encode_board(board):
    '''
    Takes board state and returns encoded 3 x board_size x board_size np array board.
    First layer contains ships, second shipyards, third halite
    '''
    
    #Do this else where and pass into fxn??
    ships=[ship.position for ship in board.ships.values()]
    owner=[ship.player_id+1 for ship in board.ships.values()]#plus 1 to begin with player 1 not 0
    shipyards=[shipyard.position for shipyard in board.shipyards.values()]
    owner2=[shipyard.player_id+1 for shipyard in board.shipyards.values()]#plus 1 to begin with player 1 not 0
    halite=[cell.halite/500 for cell in board.cells.values()]#scale 0-1 1 being max halite=500
    

    encode=np.zeros((3,board_size,board_size))#Create empty array
    
    #populate array
    for i in range(0,len(ships)):
        encode[0,ships[i][0],ships[i][1]]=owner[i]
    for i in range(0,len(shipyards)):
        encode[1,shipyards[i][0],shipyards[i][1]]=owner2[i]
    encode[2]=np.array(halite).reshape(board_size,board_size).T #Halite list reshaped into array
    


    return encode
    

In [76]:
def augment_board(encoded, centerpiece):
    '''
    Centerpiece: Ship or shipyard to be placed in bottom-right center
    Encoded: Encoded board
    Returns centered board
    '''
#    y_shift= int(board_size/2 - centerpiece[0])
#    x_shift= int(board_size/2 - centerpiece[1])

    y_shift= int(encoded.shape[1]/2 - centerpiece[0]) #Done without global varibale board_size
    x_shift= int(encoded.shape[1]/2 - centerpiece[1]) #Done without global variable board_size
    
    print(y_shift)
    augment = np.roll(encoded,y_shift,1)
    augment = np.roll(augment,x_shift,2) 
    return augment

In [77]:
encoded=encode_board(board)

myships=[ship.position for ship in me.ships]
myshipyards=[shipyards.position for ship in me.shipyards]


#Test loop just prints boards
for i in myships:
    final = augment_board(encoded, i)
    print (final)
    
#Prints nothing in this case because no shipyards
for i in myshipyards:
    final = augment_board(encoded, i)
    print (final)

3
[[[4.    0.    0.    0.    0.    2.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [3.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]]

 [[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0. 